# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
#alt.data_transformers.enable('data_server')
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [2]:
start_date = pd.to_datetime("2020-04-13")

In [3]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

,poblacion,casos_confirmados,fecha,region_id,region,provincia_id,provincia,comuna_id,comuna,tasa
0,247552.0,270.0,2020-04-27,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,109.067994
1,247552.0,9138.0,2020-10-12,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,3691.345657
2,247552.0,596.0,2020-05-29,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,240.757497
3,247552.0,6131.0,2020-08-17,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,2476.651370
4,247552.0,353.0,2020-05-11,15.0,Arica y Parinacota,151.0,Arica,15101.0,Arica,142.596303


In [4]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

,region,codigo_region,comuna,codigo_comuna,poblacion,fecha,casos_activos
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-04-13,88.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-04-13,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-04-13,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-04-13,0.0
4,Tarapaca,1,Alto Hospicio,1107.0,129999.0,2020-04-13,8.0


In [5]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

,region,fecha,fallecidos
374,Arica y Parinacota,2020-04-13,1.0
375,Tarapacá,2020-04-13,0.0
376,Antofagasta,2020-04-13,1.0
377,Atacama,2020-04-13,0.0
378,Coquimbo,2020-04-13,0.0


In [6]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.tail(20)

,grupo_de_edad,fecha,fallecidos
1639,40-49,2020-11-29,517
1640,50-59,2020-11-29,1541
1641,60-69,2020-11-29,3240
1642,70-79,2020-11-29,4319
1643,80-89,2020-11-29,3970
1644,>=90,2020-11-29,1424
1645,0-39,2020-11-30,350
1646,40-49,2020-11-30,519
1647,50-59,2020-11-30,1546
1648,60-69,2020-11-30,3250


## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos a la fecha por cada grupo etáreo.

In [7]:
alt.Chart(fallecidos_etareo).mark_bar(color="lightslategray").encode(
    x = alt.X("grupo_de_edad", axis = alt.Axis(title = "Grupo de edad", labelAngle=0)),
    y = alt.Y("fallecidos", axis = alt.Axis(title = "Fallecidos"))
).properties(
    title = "Fallecidos por COVID-19 a la fecha", 
    width = 600, 
    height = 500
)

alt.Chart(...)

**Comentarios:** Se observa que, a la fecha, la mayor cantidad de fallecidos por COVID-19 se encuentra en el rango etareo de 70-79, seguido por el grupo de edad de 80-89 y 60-69. Era de esperar que a mayor grupo de edad, mayor cantidad de muertes, algo que no sucede para el grupo etario de mayor a 90 años. Esto puede deberse a que existe poca población en Chile en ese rango de edad, lo que puede verse reflejado en una menor cantidad de muertos. Aun así, se se calculara el porcentaje respecto al total de poblacion en ese grupo de edad, probablemente el grupo de edad mayor a 90 años superaría al resto.

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

Se prepara un dataframe `poblacion_comunas` para utilizar y graficar

In [8]:
poblacion_comunas=(
    confirmados
    .groupby(["comuna","region"])
    .agg(poblacion=("poblacion","mean"))
    .reset_index()
    .sort_values(
        by="poblacion",
        ascending=False
    )
) # Será necesario colocar color de acuerdo a la región?

Se realiza el gráfico

In [36]:
alt.Chart(poblacion_comunas).mark_bar().encode(
    y = alt.Y("comuna:N", axis = alt.Axis(title = "Comuna"), sort="-x"),
    x = alt.X("poblacion", axis = alt.Axis(title = "Poblacion"))
).properties(
    title = "Poblacion por comunas de Chile",
    width = 600,
    height = 2900
)

alt.Chart(...)

**Comentarios:** Del gráfico se observa que la población es muy variable entre comunas de Chile. Lideran el ranking las comunas de Santiago, seguidas por las comunas de la región de Valparaíso. Existe gran cantidad de comunas (mas de la mitad) que poseen menos de 50.000 habitantes

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [10]:
alt.Chart(fallecidos_etareo).mark_area().encode(
    x=alt.X("yearmonthdate(fecha)", axis=alt.Axis(title="Fecha")),
    y=alt.Y("fallecidos", axis=alt.Axis(title="Fallecidos")),
    color=alt.Color("grupo_de_edad", legend=alt.Legend(title="Grupo de edad"), scale=alt.Scale(scheme="set3"))
).properties(
    title="Evolución de fallecimientos por grupo etáreo", 
    width=600, 
    height=300
)

alt.Chart(...)

**Comentarios:** Con un gráfico de área es muy fácil ver la cantidad total de fallecidos y como estos se reparten entre los rangos etarios. Se logra observar que la mayor cantidad de fallecidos se encuentra en el grupo de edad de 70-79, 80-89 y es seguido por 60-69. Esto va acorde a lo que se dice de la pandemia: La tercera edad es el grupo de riesgo que mas debe cuidarse. Los mayores a 90 años no presentan tantos fallecidos, probablemente porque no haya mucha población en Chile con esa edad. Se observa que a mediados de Julio existe un salto en el grupo de fallecidos, donde aumentó de gran forma la cantidad, lo que puede deberse a la incorporación de cifras que no habían sido incluidas por el ministerio de salud por esas fechas, tanto en fallecidos como en casos nuevos de coronavirus. 

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

Preparamos un dataframe de fallecidos `fall_graph`, de casos activos `act_graph` y casos confirmados `conf_graph` que represente la suma de cada fallecido, caso activo, caso confirmado para una misma fecha, es decir, se hace un

In [11]:
fall_graph=(
    alt.Chart(
        fallecidos
        .drop(fallecidos[fallecidos.region == "Total"].index)
        .groupby("fecha")
        .agg(fallecidos=("fallecidos","sum"))
        .reset_index()
    ).mark_line(color="burlywood").encode(
        x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
        y=alt.Y("fallecidos", axis=alt.Axis(title="Fallecidos"))
    ).properties(
        title="Evolución nacional de fallecimientos por COVID-19", 
        width=600, 
        height=300
    )
)

act_graph=(
    alt.Chart(
        activos
        .groupby("fecha")
        .agg(casos_activos=("casos_activos","sum"))
        .reset_index()
    ).mark_line(color="indigo").encode(
        x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
        y=alt.Y("casos_activos", axis=alt.Axis(title="Casos activos"))
    ).properties(
        title="Evolución nacional de casos activos de COVID-19", 
        width=600, 
        height=300
    )
)

conf_graph=(
    alt.Chart(
        confirmados
        .groupby("fecha")
        .agg(casos_confirmados=("casos_confirmados","sum"))
        .reset_index()
    ).mark_line(color="firebrick").encode(
        x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
        y=alt.Y("casos_confirmados", axis=alt.Axis(title="Casos confirmados"))
    ).properties(
        title="Evolución nacional de casos confirmados de COVID-19", 
        width=600, 
        height=300
    )
)

conf_graph & fall_graph & act_graph

alt.VConcatChart(...)

**Comentarios:** Se observa que existe una correlación entre fallecidos y casos confirmados. Ambas curvas presentan una mayor pendiente entre los meses de junio y julio correspondientes al peak de coronavirus en Chile. Luego de estas fechas, las curvas tienden a estabilizarse en un único valor de pendiente positiva, aunque de menor valor que las pendientes vistas en el peak. Los casos activos también presentan una curva con un valor máximo entre los meses de junio y julio debido al gran aumento de los csasos confirmados durante esos meses.

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

Veamos como nos va con un gráfico de líneas

In [12]:
alt.Chart(
    confirmados
    .groupby(["region","fecha"])
    .apply(lambda x: x["casos_confirmados"].sum()*100000/x["poblacion"].sum())
    .reset_index(name="tasa_region")
).mark_line().encode(
    x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
    y=alt.Y("tasa_region", axis=alt.Axis(title="Tasa de incidencia")),
    color=alt.Color("region", legend=alt.Legend(title="Región"))
).properties(
    title="Evolución de la tasa de incidencia por región", 
    width=600, 
    height=300
)

alt.Chart(...)




Ahora con un mapa de calor

In [13]:
alt.Chart(
    confirmados
    .groupby(["region","fecha"])
    .apply(lambda x: x["casos_confirmados"].sum()*100000/x["poblacion"].sum())
    .reset_index(name="tasa_region")
).mark_rect().encode(
    x=alt.X("yearmonthdate(fecha):N", axis=alt.Axis(title="Fecha",labelAngle=-60)),
    y=alt.Y("region", axis=alt.Axis(title="Region")),
    color=alt.Color("tasa_region", legend=alt.Legend(title="Tasa por región"))
).properties(
    title="Evolución de la tasa de incidencia por región", 
    width=700, 
    height=300
)

alt.Chart(...)

**Comentarios:** El mapa de calor es mucho mas claro y ordenado, es de lectura mas agradable. El gráfico de líneas serviría para identificar de forma mas específica los valores de tasas de incidencia particulares, aunque asociados principalmente a las regiones con mayores valores de tasa, ya que las otras no son claramante distinguibles. Para comparar entre regiones, el mapa de calor es mucho mejor. Se observa claramente que regiones poseen una tasa de incidencia alta y cual ha sido su evolución. Así vemos que la primera región en tener altos valores de tasa de incidencia, y que "lideró" la pandemia es la región Metropolitana. En este último tiempo, se ve que la tasa de incidencia mas alta del país está en la región de Magallanes y de la Antártica Chilena, tal cual se ha visto en la realidad nacional, siendo esta una región crítica. Les siguen a la región Metropolitana y Magallanes las regiones de Arica y Parinacota, Tarapacá, Antofagasta y Atacama. 

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [14]:
april_chart=(
    alt.Chart(
        confirmados[(confirmados["fecha"]=="2020-04-13")]
    ).mark_point().encode(
        x=alt.X("tasa", axis=alt.Axis(title="Tasa de incidencia")),
        y=alt.Y("casos_confirmados", axis=alt.Axis(title="Casos confirmados")),
        color=alt.Color("region", legend=alt.Legend(title="Región")),
        size=alt.Size("poblacion", scale=alt.Scale(range=[10, 500]))
    ).properties(
        title="Casos confirmados versus Tasa de incidencia por comuna para el 13 de Abril del 2020",
        width=600,
        height=500
    )
)

november_chart=(
    alt.Chart(
        confirmados[(confirmados["fecha"]=="2020-11-06")]
    ).mark_point().encode(
        x=alt.X("tasa", axis=alt.Axis(title="Tasa de incidencia")),
        y=alt.Y("casos_confirmados", axis=alt.Axis(title="Casos confirmados")),
        color=alt.Color("region", legend=alt.Legend(title="Región")),
        size=alt.Size("poblacion", scale=alt.Scale(range=[10, 500]))
    ).properties(
        title="Casos confirmados versus Tasa de incidencia por comuna para el 06 de Noviembre del 2020",
        width=600,
        height=500
    )
)

april_chart & november_chart

alt.VConcatChart(...)

**Comentarios:** Si los casos confirmados son muy altos, pero la tasa de incidencia es baja (puntos mas a la "izquierda") significa que muy poca población ha sido infectada con COVID, por lo que no habría tanta alarma, es de esperar que en poblaciones altas, haya una mayor cantidad de casos confirmados. Por otra parte, si la tasa de incidencia es alta, y los casos confirmados son pocos, quiere decir gran parte de la población de esa comuna ha sido infectada con COVID, lo cual es alarmante. Ahora, las comunas de la región Metropolitana son las mas visibles debido a su gran cantidad de casos confirmados, asociados a la vez a su alta población. Colocar el tamaño de los puntos proporcional a la población ayuda a visualizar el gráfico, pero no es de gran ayuda. Es posible notar que los puntos de menor tamaño son aquellos con menor cantidad de casos confirmados y con mayor tasa de incidencia. Asímismo, se observa que los puntos de mayor tamaño son los con mayor cantidad de casos confirmados y menor tasa de incidencia, asociada la mayor cantidad de población. Ayuda a la visualización, pero no es tan útil. 
Con respecto a las fechas, se nota que la magnitud entre el mes de abril y noviembre varia notoriamente. Hay mayor tasa de incidencia y casos confirmados en noviembre que en abril. También, para abril, la nube de puntos se "aplanó", lo que hablá que la tasa de incidencia aumenta con el paso del tiempo, pero no así los casos confirmados, al menos no al mismo ritmo. 

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [15]:
alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
    y=alt.Y("casos_activos", axis=alt.Axis(title="Casos activos")),
    color=alt.Color("comuna", legend=alt.Legend(title="Comuna"))
).properties(
    title="Evolución de casos activos por comuna", 
    width=600, 
    height=300
)

alt.Chart(...)

In [16]:
alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha", axis=alt.Axis(title="Fecha")),
    y=alt.Y("casos_activos", axis=alt.Axis(title="Casos activos")),
    color=alt.Color("comuna", legend=alt.Legend(title="Comuna"))
).properties(
    title="Evolución de casos activos por comuna",
    width=250,
    height=250
).facet(
    facet=alt.Facet("region", title="Región"),
    columns=2
).resolve_scale(
    y="independent", # El eje y se deja independiente para que cada gráfico se ajuste a sus datos y no se vea alterado por los altos valores de otros gráficos.
    x="independent", # El eje x independiente es solo para que se muestre el eje en cada subgráfico del facet, si no se coloca, solo se ve al final y no queda una visualización amigable
    color="independent" # Para que solo muestre la leyenda de las comunas asociadas a esa región
)

alt.FacetChart(...)

**Comentarios:** Graficar todas las comunas en un solo gráfico permite ver la evolución nacional de los casos activos e identificar fácilmente cuales fueron las comunas en donde existe una mayor cantidad de ellos. Permite además ver en forma rápida la fecha en que se dió el peak de casos activos y comparar si se dio en la misma fecha para todas las comunas o no. Se puede observar si existen otras fechas críticas y en qué comunas se dio (si existen otros picks fuera de la fecha del pick nacional). Los contras, es que no se puede observar al detalle las comunas con poca cantidad de casos activos, que generalmente concuerdan con las comunas que poseen baja población total. Además, dada la cantidad de comunas, queda muy llena de información y es de dificil lectura.
Separar los gráficos por regiones entrega información mas detallada de las comunas. Se puede observar que comunas dentro de la región son mas críticas y permite escalar los ejes para obtener valores concretos. Permite, en palabras simples, observar la realidad de cada comuna y la fecha en donde se dan los picks.


## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [17]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}

region_names2 = {
    "Valparaiso": "Valparaíso",
    "Metropolitana": "Metropolitana de Santiago",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Biobio": "Biobío",
    "La Araucania": "La Araucanía",
    "Los Rios": "Los Ríos",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena",
    "Nuble": "Ñuble",
    "Tarapaca": "Tarapacá",
}
fallecidos["region"]=fallecidos["region"].replace(region_names)
activos["region"] = activos["region"].replace(region_names2)

In [18]:
activos_mod = activos.groupby("region").agg(prom_activos=("casos_activos","mean")).reset_index()
fallecidos_mod=fallecidos.groupby("region").agg(max_fallecidos=("fallecidos", "max")).reset_index()
confirmados_mod=confirmados.groupby("region").agg(max_confirmados=("casos_confirmados", "max")).reset_index()
tabla_region=(
    confirmados_mod
    .merge(activos_mod, 
           how="inner", 
           on="region")
    .merge(fallecidos_mod, 
           how="inner", 
           on="region")
).rename(
    columns={
        "prom_activos":"Casos activos promedio",
        "max_fallecidos":"Máxima cantidad de fallecidos",
        "max_confirmados":"Máxima cantidad de casos confirmados"
    }
)

#Llevamos la tabla a un "long-form" de acuerdo a la documentación de altair, que dice que funciona mejor

tabla_region=tabla_region.melt(
    id_vars="region", 
    value_vars=[
        "Casos activos promedio", 
        "Máxima cantidad de fallecidos", 
        "Máxima cantidad de casos confirmados"
    ], 
    var_name="dato", 
    value_name="valor"
)

In [19]:
alt.Chart(tabla_region).mark_bar().encode(
    x = alt.X("dato", axis = None, sort=["Máxima cantidad de casos confirmados","Máxima cantidad de fallecidos","Casos activos promedio"]),
    y = alt.Y("valor", axis = alt.Axis(title = "N° de personas")),
    color=alt.Color("dato", legend=alt.Legend(title="Categoría"), scale=alt.Scale(scheme="set1")),
    column=alt.Column("region", title="Región")
).properties(
    title="Máximo de casos confirmados, fallecidos y promedio de casos activos por Región",
    width = 120, 
    height = 300
).configure_title(
    align="center",
    anchor="middle",
    fontSize=20
)
#alt.Chart(tabla_region).mark_bar().encode(
#    x = alt.X("dato", axis = alt.Axis(title = "hola")),
#    y = alt.Y("valor", axis = alt.Axis(title = "N° de personas")),
#    color=alt.Color("dato", legend=alt.Legend(title="Detalle"))
#).properties(
#    title = "Fallecidos por COVID-19 a la fecha", 
#    width = 120, 
#    height = 300
#).facet(
#    facet=alt.Facet("region", title="Región"),
#    columns=4
#).resolve_scale(
#    y="independent", # El eje y se deja independiente para que cada gráfico se ajuste a sus datos y no se vea alterado por los altos valores de otros gráficos.
#    x="independent" # El eje x independiente es solo para que se muestre el eje en cada subgráfico del facet, si no se coloca, solo se ve al final y no queda una visualización amigable
#)

alt.Chart(...)

**Comentarios:** Con el gráfico se puede observar de forma clara la diferencia de casos confirmados, activos y fallecidos entre regiones. Se nota de forma clara cuales son las regiones mas y menos afectadas, aunque siempre será importante saber cual es su relación con la población. Por ejemplo, es de esperar que la región Metropolitana tenga un mayor número de casos, en cualquier categoría, dada su gran población. Hay regiones que prácticamente no se ven tan afectadas y que poseen un número ínfimo de casos activos, lo que puede ayudar para tomar decisiones en cuanto a cuarentenas. También, dentro de una misma región se puede observar las proporciones de casos confirmados y fallecidos, para conocer su tasa de mortalidad. Por ejemplo, la región metropolitana posee 1/3 de fallecidos con respecto a casos confirmados

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [20]:
import geopandas as gpd
from pathlib import Path

In [21]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

,Region,objectid,cir_sena,codregion,area_km,st_area_sh,st_length_,geometry
0,Región Metropolitana de Santiago,1092,7,13,15392.030737,2.225204e+10,1.064253e+06,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,Región de Antofagasta,1086,3,2,126071.431981,1.508452e+11,2.516112e+06,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Región de Arica y Parinacota,1084,1,15,16866.819844,1.886869e+10,7.505296e+05,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,Región de Atacama,1089,4,3,75661.248635,9.643906e+10,2.401741e+06,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,Región de Aysén del Gral.Ibañez del Campo,1088,14,11,106703.377369,2.242743e+11,4.144481e+07,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [22]:
type(regiones)

geopandas.geodataframe.GeoDataFrame

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [23]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

alt.Chart(...)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [24]:
confirmados_p1 = (
    confirmados
    .groupby(["region","region_id","fecha"])
    .apply(lambda x: x["casos_confirmados"].sum()*100000/x["poblacion"].sum())
    .reset_index(name="tasa")
).merge(
    confirmados
    .groupby(["region","region_id","fecha"])
    .agg(casos_confirmados=("casos_confirmados","sum"),
        poblacion=("poblacion","sum"))
    .reset_index(),
    how="inner",
    on=["region_id", "fecha","region"]
)
confirmados_p1

,region,region_id,fecha,tasa,casos_confirmados,poblacion
0,Antofagasta,2.0,2020-04-13,21.536336,149.0,691854.0
1,Antofagasta,2.0,2020-04-15,25.438893,176.0,691854.0
2,Antofagasta,2.0,2020-04-17,30.497764,211.0,691854.0
3,Antofagasta,2.0,2020-04-20,38.158340,264.0,691854.0
4,Antofagasta,2.0,2020-04-24,53.190413,368.0,691854.0
...,...,...,...,...,...,...
1051,Ñuble,16.0,2020-11-13,2064.701271,10562.0,511551.0
1052,Ñuble,16.0,2020-11-16,2103.602573,10761.0,511551.0
1053,Ñuble,16.0,2020-11-20,2154.623879,11022.0,511551.0
1054,Ñuble,16.0,2020-11-23,2197.239376,11240.0,511551.0


In [25]:
activos_p1 = (
    activos
    .groupby(["region", "codigo_region", "fecha"])
    .agg(casos_activos=("casos_activos","sum"))
    .reset_index()
    .rename(columns={"codigo_region":"region_id"})
    .astype({"region_id": np.float})
).drop(
    columns="region"
)
activos_p1

,region_id,fecha,casos_activos
0,2.0,2020-04-13,78.0
1,2.0,2020-04-15,88.0
2,2.0,2020-04-17,103.0
3,2.0,2020-04-20,129.0
4,2.0,2020-04-24,172.0
...,...,...,...
1051,16.0,2020-11-13,390.0
1052,16.0,2020-11-16,429.0
1053,16.0,2020-11-20,476.0
1054,16.0,2020-11-23,492.0


In [26]:
fallecidos_p1 = (
    fallecidos
    .groupby(["region","fecha"])
    .agg(fallecidos=("fallecidos","sum"))
    .reset_index()
)
fallecidos_p1
#Tiene mas filas porque hay mas fechas, pero en el merge se corregirá y se dejará de lado las fechas que no sirvan

,region,fecha,fallecidos
0,Antofagasta,2020-04-13,1.0
1,Antofagasta,2020-04-14,1.0
2,Antofagasta,2020-04-15,1.0
3,Antofagasta,2020-04-16,1.0
4,Antofagasta,2020-04-17,1.0
...,...,...,...
3956,Ñuble,2020-11-27,176.0
3957,Ñuble,2020-11-28,176.0
3958,Ñuble,2020-11-29,177.0
3959,Ñuble,2020-11-30,178.0


In [27]:
regiones_p1 = (
    regiones
    .drop(columns=["objectid","cir_sena","area_km","st_area_sh","st_length_"])
    .rename(columns={"codregion":"region_id","Region":"region"})
    .astype({"region_id": np.float})
    .drop(columns="region")
)
regiones_p1.head()

,region_id,geometry
0,13.0,"POLYGON ((-7873736.745 -3885505.642, -7873695...."
1,2.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,15.0,"POLYGON ((-7727277.278 -1997230.768, -7726464...."
3,3.0,"MULTIPOLYGON (((-7900342.628 -3153340.296, -79..."
4,11.0,"MULTIPOLYGON (((-8208500.834 -5733817.475, -82..."


In [28]:
casos_geo = (
    confirmados_p1
    .merge(
        activos_p1,
        how="inner",
        on=["region_id","fecha"]
    ).merge(
        fallecidos_p1,
        how="left",
        on=["fecha","region"]
    ).merge(
        regiones_p1,
        how="inner",
        on="region_id"
    )
)

casos_geo

,region,region_id,fecha,tasa,casos_confirmados,poblacion,casos_activos,fallecidos,geometry
0,Antofagasta,2.0,2020-04-13,21.536336,149.0,691854.0,78.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
1,Antofagasta,2.0,2020-04-15,25.438893,176.0,691854.0,88.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
2,Antofagasta,2.0,2020-04-17,30.497764,211.0,691854.0,103.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
3,Antofagasta,2.0,2020-04-20,38.158340,264.0,691854.0,129.0,1.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
4,Antofagasta,2.0,2020-04-24,53.190413,368.0,691854.0,172.0,2.0,"MULTIPOLYGON (((-7874671.129 -2977676.850, -78..."
...,...,...,...,...,...,...,...,...,...
1051,Ñuble,16.0,2020-11-13,2064.701271,10562.0,511551.0,390.0,162.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1052,Ñuble,16.0,2020-11-16,2103.602573,10761.0,511551.0,429.0,163.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1053,Ñuble,16.0,2020-11-20,2154.623879,11022.0,511551.0,476.0,168.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."
1054,Ñuble,16.0,2020-11-23,2197.239376,11240.0,511551.0,492.0,172.0,"POLYGON ((-8053806.540 -4431488.942, -8053866...."


Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [29]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [30]:
def covid_chile_chart(fecha, col):
    
    fecha = pd.to_datetime(fecha)
    data = casos_geo[casos_geo["fecha"]==fecha]
    
    chart = alt.Chart(data).mark_geoshape().encode(
        color=col
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [31]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

alt.Chart(...)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [32]:
import ipywidgets as widgets
from ipywidgets import interactive, interact
import datetime

In [33]:
col_widget = widgets.Dropdown(options=["poblacion","casos_confirmados","tasa","casos_activos","fallecidos"],
                             continuous_update=False,
                              value="tasa",
                             description="Columna")

In [34]:
fecha_widget = widgets.DatePicker(description="Fecha",
                                 continuous_update=False)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [35]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

interactive(children=(DatePicker(value=None, description='Fecha'), Dropdown(description='Columna', index=2, op…

**Comentarios:** Es muy interesante poder ver como varían espacialmente las distintas categorías (poblacion, casos activos, tasa, etc), además de su variación espacial al mismo tiempo. Si bien el proceso de carga es un poco lento, la visualización geográfica puede ayudar mucho a una persona que le cueste la lectura de gráficos. Es un buen mapa para colocar en alguna página de acceso público, para poder ver los indicadores de COVID a lo largo de la nación. 